In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np

import commons
import utils
# from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import  cleaned_text_to_sequence
from utils import load_wav_to_torch
from mel_processing import spectrogram_torch
from scipy.io.wavfile import write



config_path = 'configs/config.json'
hps = utils.get_hparams_from_file(config_path)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

utils.load_checkpoint(utils.latest_checkpoint_path("logs/snac/", "G_*.pth"), net_g,
                      None, None)
1

/home/ubuntu/miniconda3/envs/t/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logs/snac/G_24800.pth
load 
INFO:root:Loaded checkpoint 'logs/snac/G_24800.pth' (iteration 12)


1

In [ ]:
stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([4]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [3]:
dev = "cuda"
net_g = net_g.to(dev)

In [9]:
import commons
text = "tʃ⁼iŋ→t⁼ʊŋ→ %p⁼aɪ↓↑fən→ts`⁼ɹ`→ %p⁼a→s`ɹ`↑t⁼ə %jʊŋ↓xu↓ %ɥ↓↑tʰa→ %liɑʊ↑tʰjɛn→ %p⁼aɪ↓↑fən→ts`⁼ɹ`→ %əɹ`↓s`ɹ`↑ %jʊŋ↓ɥ↑ %maɪ↓↑t⁼ʊŋ→ʃi $|0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0"
text="a, i↓idesɯ*ne. o↑nnanokoɾaʃi↓saga gɯ↓t#to ma↑ʃi*te i↑ma↓sɯ*jo|1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1"
text="wa↑taʃidat#te be↑tsɯn^i, ka↑keɾano ka↑iʃɯɯdakeo mo↑kɯ*tekin^i ʃi*↑te i↑ɾɯ wa↑kedewaaɾimase↓Njo|1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1"
cleaned_text, lang = text.split("|")
lang = [int(i) for i in lang.split(" ")]
text_norm = cleaned_text_to_sequence(cleaned_text)
if hps.data.add_blank:
    text_norm, lang = commons.intersperse_with_language_id(text_norm, lang, 0)

text_norm = torch.LongTensor(text_norm)
lang = torch.LongTensor(lang)
x_tst = text_norm.to(dev).unsqueeze(0)
lang = lang.to(dev).unsqueeze(0)
x_tst_lengths = torch.LongTensor([text_norm.size(0)]).to(dev)
speaker_id = torch.LongTensor([0]).to(dev)

In [13]:
for i in range(4):
    with torch.no_grad():
        y_hat,_,_,_ =net_g.infer(x_tst, x_tst_lengths, lang,None,
                                       sid=speaker_id,  predict_style=True, style_noise_scale=1.4)
        audio = y_hat[0, :, :].cpu().numpy()
        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
